# Interface plate for Newport M-EN86 Lab Jack

In [ ]:
from build123d import *
import scrollcase as sc
import logging
from functools import partial
from ocp_vscode import *
from dataclasses import dataclass
from meshlib import mrmeshpy as mm
from pathlib import Path

logging.basicConfig()  # Required in Jupyter to correctly set output stream
logging.getLogger("scrollcase").setLevel(logging.DEBUG)

In [ ]:
@dataclass
class interface_specs:
    plate_width_mm: float = 140
    plate_depth_mm: float = 10

    box_width_mm: float = 12.5
    box_height_mm: float = 5

    captive_nut_holder_outer_diam_mm: float = 135
    captive_nut_holder_inner_diam_mm: float = 115
    captive_nut_holder_height_mm: float = 10
    captive_nut_holder_width_mm: float = 50

    captive_nut_slot_width_mm: float = 13
    captive_nut_slot_depth_mm: float = 6.25
    captive_nut_slot_distance_apart_mm: float = 117.5

    bolt_hole_diameter_mm: float = 6.8

    m4_clearance_diameter_mm: float = 4.7
    m4_recess_depth_mm: float = 4
    m4_recess_diameter_mm: float = 8

    pattern_widths = [63, 77]


specs = interface_specs()

def interface_plate(specs: interface_specs):
    with BuildPart() as part:
        # Base plate
        Box(
            specs.plate_width_mm,
            specs.plate_width_mm,
            specs.plate_depth_mm,
            align=(Align.CENTER, Align.CENTER, Align.MAX),
        )

        # Cylinder on top of the plate
        Cylinder(
            radius=specs.captive_nut_holder_outer_diam_mm / 2,
            height=specs.captive_nut_holder_height_mm,
            align=(Align.CENTER, Align.CENTER, Align.MIN),
        )

        # Subtract inner cylinder
        Cylinder(
            radius=specs.captive_nut_holder_inner_diam_mm / 2,
            height=specs.captive_nut_holder_height_mm,
            align=(Align.CENTER, Align.CENTER, Align.MIN),
            mode=Mode.SUBTRACT,
        )

        # Captive nut holders
        with Locations((specs.captive_nut_holder_width_mm / 2, 0, 0)):
            Box(
                specs.plate_width_mm,
                specs.plate_width_mm,
                specs.captive_nut_holder_height_mm,
                align=(Align.MIN, Align.CENTER, Align.MIN),
                mode=Mode.SUBTRACT,
            )

        with Locations((-specs.captive_nut_holder_width_mm / 2, 0, 0)):
            Box(
                specs.plate_width_mm,
                specs.plate_width_mm,
                specs.captive_nut_holder_height_mm,
                align=(Align.MAX, Align.CENTER, Align.MIN),
                mode=Mode.SUBTRACT,
            )

        # Captive nut slots
        with Locations((0, specs.captive_nut_slot_distance_apart_mm / 2, 0)):
            Box(
                specs.captive_nut_slot_width_mm,
                specs.captive_nut_slot_depth_mm,
                specs.captive_nut_holder_height_mm,
                align=(Align.CENTER, Align.MIN, Align.MIN),
                mode=Mode.SUBTRACT,
            )

        with Locations((0, -specs.captive_nut_slot_distance_apart_mm / 2, 0)):
            Box(
                specs.captive_nut_slot_width_mm,
                specs.captive_nut_slot_depth_mm,
                specs.captive_nut_holder_height_mm,
                align=(Align.CENTER, Align.MAX, Align.MIN),
                mode=Mode.SUBTRACT,
            )

        # Bolt holes
        with Locations((0, 0, specs.captive_nut_holder_height_mm / 2)):
            Cylinder(
                radius=specs.bolt_hole_diameter_mm / 2,
                height=specs.plate_width_mm,
                align=(Align.CENTER, Align.CENTER, Align.CENTER),
                rotation=(90, 0, 0),
                mode=Mode.SUBTRACT,
            )

        # Alignment box
        Box(
            specs.box_width_mm,
            specs.box_width_mm,
            specs.box_height_mm,
            align=(Align.CENTER, Align.CENTER, Align.MIN),
        )

        # Interface mount holes
        for pattern_width in specs.pattern_widths:
            for x_sign in [1, -1]:
                for y_sign in [1, -1]:
                    with Locations((x_sign * pattern_width / 2, y_sign * pattern_width / 2, 0)):
                        # M4 clearance hole
                        Cylinder(
                            radius=specs.m4_clearance_diameter_mm / 2,
                            height=specs.plate_depth_mm,
                            align=(Align.CENTER, Align.CENTER, Align.MAX),
                            mode=Mode.SUBTRACT,
                        )

                        # Recess
                        Cylinder(
                            radius=specs.m4_recess_diameter_mm / 2,
                            height=specs.m4_recess_depth_mm,
                            align=(Align.CENTER, Align.CENTER, Align.MAX),
                            mode=Mode.SUBTRACT,
                        )

    return part

plate = interface_plate(specs)
show(plate, reset_camera=Camera.KEEP)

# for plate_depth in [7, 10, 12.7]:
#     specs = interface_specs(plate_depth_mm=plate_depth)

#     plate = interface_plate(specs)

#     mesh = sc.mesh.brep_to_mesh(plate.solids()[0])

#     mm.saveMesh(mesh, Path(f"plate_{plate_depth}.stl"))